In [1]:
import re
import uuid
import cairocffi as cairo
import numpy as np
from scipy import ndimage
import os
import random
import cv2
import glob
from PIL import Image, ImageFont, ImageDraw
#import textwrap
import sys
import string

In [2]:
def add_pad(img, pad=1, color=(0, 0, 0)):
    width, height = img.size
    new_width = width + pad + pad
    new_height = height + pad + pad
    result = Image.new(img.mode, (new_width, new_height), color)
    result.paste(img, (pad, pad))
    return result

In [3]:
def text_to_image(text: str, font_filepath: str, font_size: int, color=(0, 0, 0), font_align="center"):
    font = ImageFont.truetype(font_filepath, size=font_size)
    
    # Calculate size of text box
    lines = text.split('\n')
    max_width = max(font.getbbox(line)[2] for line in lines)
    total_height = sum(font.getbbox(line)[3] - font.getbbox(line)[1] for line in lines)
    
    img = Image.new("RGB", (max_width, total_height), color=(255, 255, 255))
    img = add_pad(img, pad=5, color=(255, 255, 255))
    
    draw = ImageDraw.Draw(img)
    y_text = 0
    for line in lines:
        bbox = font.getbbox(line)
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        if font_align == "center":
            draw.text(((max_width - width) / 2, y_text), line, font=font, fill=color)
        elif font_align == "left":
            draw.text((0, y_text), line, font=font, fill=color)
        elif font_align == "right":
            draw.text((max_width - width, y_text), line, font=font, fill=color)
        y_text += height
    
    img = add_pad(img, pad=5, color=(255, 255, 255))
    img = np.asarray(img)
    return img

In [4]:
def paint_text(text, multi_fonts=False, mode="multiple"):
    surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, 1, 1)
    context = cairo.Context(surface)
    rand_font_size = np.random.randint(25, 40)
    
    if mode == "multiple":
        num = random.randint(0, 10)
        if num <= 3:
            mode = "normal"
        elif num <= 5:
            mode = "normal_fonts_path"
        elif num > 5:
            mode = "other_fonts"
    
    normal_fonts = ['Arial', 'Calibri', 'Times New Roman', 'Verdana', 'Andale Mono', 'Georgia', 'Trebuchet MS']
    normal_fonts_path = glob.glob("normal_fonts/*")
    other_fonts = glob.glob("other_fonts/*")
    
    if mode == "normal":
        fonts = normal_fonts
        fonts_tuple = (np.random.choice(fonts),
                       np.random.choice([cairo.FONT_SLANT_ITALIC, cairo.FONT_SLANT_OBLIQUE]),
                       np.random.choice([cairo.FONT_WEIGHT_BOLD, cairo.FONT_WEIGHT_NORMAL]))
        if multi_fonts:  
            context.select_font_face(*fonts_tuple)
        else:
            context.select_font_face('Arial', cairo.FONT_SLANT_ITALIC, cairo.FONT_WEIGHT_NORMAL)
        
        context.set_font_size(rand_font_size)
        x_bearing, y_bearing, width, height, x_advance, y_advance = context.text_extents(text)[:6]

        h, w = int(height) + 10, int(width) + 10
        surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, w, h)
        
        with cairo.Context(surface) as context:
            context.set_font_size(rand_font_size)
            color_background = 255
            context.set_source_rgb(color_background, color_background, color_background)  # White
            context.paint()
            if multi_fonts:
                context.select_font_face(*fonts_tuple)
            else:
                context.select_font_face('Arial', cairo.FONT_SLANT_ITALIC, cairo.FONT_WEIGHT_NORMAL)
            
            max_shift_x = 10
            max_shift_y = 10
            top_left_x = np.random.randint(0, int(max_shift_x))
            top_left_y = np.random.randint(0, int(max_shift_y))
        
            context.move_to(top_left_x - int(x_bearing), top_left_y - int(y_bearing))
            color_text = 0
            context.set_source_rgb(color_text, color_text, color_text)
            context.show_text(text)
        
        buf = surface.get_data()
        array = np.ndarray((surface.get_height(), surface.get_width(), 4), buffer=buf, dtype=np.uint8)
        image_proc = array[..., :3]
    
    elif mode == "normal_fonts_path":
        font_filepath = random.choice(normal_fonts_path)
        image_proc = text_to_image(text, font_filepath, rand_font_size)
    
    elif mode == "other_fonts":
        font_filepath = random.choice(other_fonts)
        image_proc = text_to_image(text, font_filepath, rand_font_size)
    
    image_proc = cv2.resize(image_proc, (28, 28))
    image_proc = cv2.cvtColor(image_proc, cv2.COLOR_RGB2GRAY)
    
    return image_proc

In [5]:
alphabet = list(string.ascii_uppercase) + list(map(str, range(10)))

In [6]:
for each_class in alphabet:
    class_name = str(each_class)
    if not os.path.exists(f"Dataset/{class_name}"):
        os.mkdir(f"Dataset/{class_name}")
    for i in range(2000):
        image_proc = paint_text(class_name, multi_fonts=True, mode="multiple")
        cv2.imwrite(f"Dataset/{class_name}/data_{i}.png", image_proc)